In [1]:
import math
import collections
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import utils

In [2]:
class Simulation:
    def __init__(self, scenarios):
        self.scenarios = scenarios
        self.history = pd.DataFrame()
        self.profit = pd.DataFrame(columns=['scenario_name', 'profit_real_estate', 'profit_stocks'])

    def simulate(self):
        profit_data = []
        for i, scenario in enumerate(self.scenarios[:100]):
            scenario.run()
            self.history = self.history.append(scenario.history, ignore_index=True)
            profit_data.append([scenario.name, scenario.profit_real_estate, scenario.profit_stocks])
            clear_output()
            print(f'Completed scenario {i}.')
        self.profit = pd.DataFrame(profit_data, columns=['scenario_name', 'profit_real_estate', 'profit_stocks'])

In [3]:
params = {'growth_rate_real_estate': np.arange(0.01, 0.15, 0.01),
          'growth_rate_stocks': np.arange(0.01, 0.15, 0.01),
          'initial_price_stocks': [100],
          'mortgage_overpayment_amount': list(range(0, 20001, 5000)),
          'investment_amount': list(range(0, 20001, 5000))}


scenarios = []
for gre in params['growth_rate_real_estate']:
    for gs in params['growth_rate_stocks']:
        for ps in params['initial_price_stocks']:
            for ma in params['mortgage_overpayment_amount']:
                for ia in params['investment_amount']:
                    scenarios.append(utils.Scenario(utils.RealEstate(4.2e6, utils.Mortgage(0.0305, 3.6e6, 30, 12)), utils.Portfolio(), gre, gs, ps, ma, ia))


sim = Simulation(scenarios)

In [4]:
%%time
sim.simulate()

Completed scenario 99.
CPU times: user 2.28 s, sys: 165 ms, total: 2.45 s
Wall time: 2.37 s


In [5]:
data = sim.history

In [6]:
data.head()

,mortgage_amount,price_index,current_price_real_est,interest_amount,capital_downpayment,invested_amount,purchase_price_stocks,current_price_stocks,value_stocks,scenario_name,month,profit_stocks,cumulative_interest_amount,cumulative_profit_stocks,current_profit_real_estate
0,3.600000e+06,1.000000,4.200000e+06,9150.000000,6124.995854,0,100.000000,134.969018,0.0,0.01_0.01_0_0,0,0.0,9150.000000,0.0,-9150.000000
1,3.593875e+06,1.000833,4.203500e+06,9134.432302,6140.563552,0,100.083333,134.969018,0.0,0.01_0.01_0_0,1,0.0,18284.432302,0.0,-14784.432302
2,3.587734e+06,1.001667,4.207003e+06,9118.825037,6156.170818,0,100.166736,134.969018,0.0,0.01_0.01_0_0,2,0.0,27403.257339,0.0,-20400.340672
3,3.581578e+06,1.002502,4.210509e+06,9103.178102,6171.817752,0,100.250208,134.969018,0.0,0.01_0.01_0_0,3,0.0,36506.435441,0.0,-25997.683011
4,3.575406e+06,1.003338,4.214018e+06,9087.491399,6187.504455,0,100.333750,134.969018,0.0,0.01_0.01_0_0,4,0.0,45593.926840,0.0,-31576.417116


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20740 entries, 0 to 20739
Data columns (total 15 columns):
mortgage_amount               20740 non-null float64
price_index                   20740 non-null float64
current_price_real_est        20740 non-null float64
interest_amount               20740 non-null float64
capital_downpayment           20740 non-null float64
invested_amount               20740 non-null int64
purchase_price_stocks         20740 non-null float64
current_price_stocks          20740 non-null float64
value_stocks                  20740 non-null float64
scenario_name                 20740 non-null object
month                         20740 non-null int64
profit_stocks                 20740 non-null float64
cumulative_interest_amount    20740 non-null float64
cumulative_profit_stocks      20740 non-null float64
current_profit_real_estate    20740 non-null float64
dtypes: float64(12), int64(2), object(1)
memory usage: 2.4+ MB


In [8]:
# create scenario compiler function (from a dictionary of lists of parameters)
# add overall scenario results (profit stocks, profit mortgage, etc.)
# implement https://en.wikipedia.org/wiki/Internal_rate_of_return

In [12]:
sim.profit.head()

,scenario_name,profit_real_estate,profit_stocks
0,0.01_0.01_0_0,-430299.753895,0.000000e+00
1,0.01_0.01_0_5000,-430299.753895,2.998895e+05
2,0.01_0.01_0_10000,-430299.753895,5.997791e+05
3,0.01_0.01_0_15000,-430299.753895,8.996686e+05
4,0.01_0.01_0_20000,-430299.753895,1.199558e+06


In [13]:
sim.profit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
scenario_name         100 non-null object
profit_real_estate    100 non-null float64
profit_stocks         100 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB
